In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
def load_data(directory):
    data = []
    vehicle_ids = []

    for i, file_name in enumerate(os.listdir(directory)):

        # Limit elements in Tensor for performance purposes, remove this when ready for full scale training
        if(i % 5000 == 0):
            print(f'Loading {i}')

        # Extract vehicle IDs from the filename
        vehicle_id = int(file_name.split('_')[0])
        vehicle_ids.append(vehicle_id)

        # Load NPY files
        file_path = os.path.join(directory, file_name)
        loaded_data = np.load(file_path)

        data.append(loaded_data)
        
        # Reshape loaded data and append to a list
        #reshaped_data = loaded_data.reshape((1, 32, 64))
        # data.append(reshaped_data)

    # Converting data and labels to PyTorch Tensors
    data = np.array(data)
    data = torch.from_numpy(data)
    vehicle_ids = torch.tensor(vehicle_ids, dtype=torch.long)
    
    # Adjust labels to be in range 0 to 1361
    if(torch.min(vehicle_ids).item() == 1):
        vehicle_ids = vehicle_ids - 1
    
    return (data, vehicle_ids)

In [3]:
# Load train data and labels from train directory
train_directory = "vehicle-x/train"
test_directory = "vehicle-x/test"

train_data, vehicle_ids_train = load_data(train_directory)
test_data, vehicle_ids_test = load_data(test_directory)

Loading 0
Loading 5000
Loading 10000
Loading 15000
Loading 20000
Loading 25000
Loading 30000
Loading 35000
Loading 40000
Loading 45000
Loading 0
Loading 5000
Loading 10000
Loading 15000


In [4]:
# Hyperparameters
num_epochs = 100
batch_size = 32
test_batch_size = 64
learning_rate = 0.01

class CustomDataset():
    def __init__(self, data_array, labels):
        self.data = data_array
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Create custom dataset from the tensors
train_dataset = CustomDataset(train_data, vehicle_ids_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = CustomDataset(test_data, vehicle_ids_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)

In [5]:
# Define the neural network
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, output_dim),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        return self.fc(x)

# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialise network
model = SimpleClassifier(2048, 1362).to(device)

In [6]:
def train():
    model.train()
    correct = 0
    for batch_idx, (batch_data, batch_labels) in enumerate(train_loader):
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_data)

        pred = outputs.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(batch_labels.data.view_as(pred)).long().cpu().sum()
        
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 500 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], step[{batch_idx + 1}], loss: {loss.item():.4f}')

    print('Train Accuracy:{}/{}'.format(correct, len(train_loader.dataset)))

In [7]:
def test():
    model.eval()
    correct = 0

    for batch_data, batch_labels in test_loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        
        outputs = model(batch_data)
        
        pred = outputs.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(batch_labels.data.view_as(pred)).long().cpu().sum()

    print('Test accuracy:{}/{}\n'.format(correct, len(test_loader.dataset)))

In [8]:
# Train the model
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train()
    test()

Epoch [1/100], step[500], loss: 7.2180
Epoch [1/100], step[1000], loss: 7.1868
Train Accuracy:36/45438
Test accuracy:10/15142

Epoch [2/100], step[500], loss: 7.2224
Epoch [2/100], step[1000], loss: 7.2250
Train Accuracy:41/45438
Test accuracy:6/15142

Epoch [3/100], step[500], loss: 7.2261
Epoch [3/100], step[1000], loss: 7.2219
Train Accuracy:39/45438
Test accuracy:16/15142

Epoch [4/100], step[500], loss: 7.2102
Epoch [4/100], step[1000], loss: 7.2301
Train Accuracy:38/45438
Test accuracy:10/15142

Epoch [5/100], step[500], loss: 7.1902
Epoch [5/100], step[1000], loss: 7.2022
Train Accuracy:38/45438
Test accuracy:4/15142

Epoch [6/100], step[500], loss: 7.2240
Epoch [6/100], step[1000], loss: 7.2282
Train Accuracy:39/45438
Test accuracy:7/15142

Epoch [7/100], step[500], loss: 7.2179
Epoch [7/100], step[1000], loss: 7.2084
Train Accuracy:42/45438
Test accuracy:17/15142

Epoch [8/100], step[500], loss: 7.2184
Epoch [8/100], step[1000], loss: 7.2293
Train Accuracy:58/45438
Test accura